In [1]:
import tensorflow as tf
import numpy as np

In [5]:
class Autoencoder:
    def __init__(self, input_dim, hidden_dim, epoch=250, learning_rate=0.001):
        self.epoch = epoch
        self.learning_rate = learning_rate
        
        x = tf.placeholder(dtype=tf.float32, shape=[None, input_dim])
        
        with tf.name_scope('encode'):
            weights = tf.Variable(tf.random_normal([input_dim, hidden_dim]), dtype=tf.float32, name='weights')
            biases = tf.Variable(tf.zeros([hidden_dim]), name='biases')
            encoded = tf.nn.tanh(tf.matmul(x, weights) + biases)
        with tf.name_scope('decode'):
            weights = tf.Variable(tf.random_normal([hidden_dim, input_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([input_dim]), name='biases')
            decoded = tf.matmul(encoded, weights) + biases
        
        self.x = x
        self.encoded = encoded
        self.decoded = decoded
        
        self.loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.x, self.decoded))))
        
        self.train_op = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss)
        self.saver = tf.train.Saver()
        
    def train(self, data):
        num_samples = len(data)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(self.epoch):
                for j in range(num_samples):
                    l, _ = sess.run([self.loss, self.train_op], feed_dict={self.x: [data[j]]})
                if i % 10 == 0:
                    print('epoch {0}: loss = {1}'.format(i, l))
                    self.saver.save(sess, './model.ckpt')
            self.saver.save(sess, './model.ckpt')
    
    def test(self, data):
        with tf.Session() as sess:
            self.saver.restore(sess, './model.ckpt')
            hidden, reconstructed = sess.run([self.encoded, self.decoded], feed_dict={self.x: data})
            print('input', data)
            print('compressed', hidden)
            print('reconstructed', reconstructed)
            return reconstructed

In [7]:
from sklearn import datasets

hidden_dim = 1
data = datasets.load_iris().data
input_dim = len(data[0])
ae = Autoencoder(input_dim, hidden_dim)
ae.train(data)
ae.test([[8,4,6,2]])

epoch 0: loss = 3.1613223552703857
epoch 10: loss = 1.4270784854888916
epoch 20: loss = 0.6996403932571411
epoch 30: loss = 0.4325000047683716
epoch 40: loss = 0.379517138004303
epoch 50: loss = 0.37081584334373474
epoch 60: loss = 0.3666681945323944
epoch 70: loss = 0.36271238327026367
epoch 80: loss = 0.35832926630973816
epoch 90: loss = 0.3533138036727905
epoch 100: loss = 0.34747904539108276
epoch 110: loss = 0.34057411551475525
epoch 120: loss = 0.33219805359840393
epoch 130: loss = 0.32160285115242004
epoch 140: loss = 0.30815666913986206
epoch 150: loss = 0.29402828216552734
epoch 160: loss = 0.2791764438152313
epoch 170: loss = 0.265682578086853
epoch 180: loss = 0.2522948980331421
epoch 190: loss = 0.2394627183675766
epoch 200: loss = 0.23055315017700195
epoch 210: loss = 0.23047366738319397
epoch 220: loss = 0.23829425871372223
epoch 230: loss = 0.25287818908691406
epoch 240: loss = 0.2728956639766693
input [[8, 4, 6, 2]]
compressed [[-0.96406698]]
reconstructed [[ 6.43370962

array([[ 6.43370962,  2.8234024 ,  5.38293362,  1.8696965 ]], dtype=float32)